In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../code')
import compute_indicators as ci

In [3]:
import numpy as np
import pandas as pd
import talib as ta
import tqdm.notebook as tqdm

In [5]:
import alpaca_trade_api as tradeapi

key_id = None
secret_key = None
with open('../files/private/alpakey') as key_file:
    keys = key_file.readlines()
    key_id = keys[0].strip()
    secret_key = keys[1].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca = tradeapi.REST(key_id, secret_key, api_url, api_version='v2')

In [6]:
import json

symbols = []
with open('../files/public/top_symbols.json') as top_symbols:
    symbols = json.load(top_symbols)

In [7]:
columns = json.load(open('../models/columns.json', 'rb'))

In [8]:
from keras.models import load_model

model = load_model('../models/daily.h5')

Using Theano backend.


In [9]:
from pickle import load

scaler = load(open('../models/scaler.pkl', 'rb'))

In [10]:
def predict(stock, symbol):
    return ci.predict_today(stock, columns, model, scaler)

In [11]:
stocks = []

for symbol in tqdm(symbols):
    try:
        stock = alpaca.polygon.historic_agg_v2(symbol, 1, 'day',
                                               _from='2019-01-01', to='2021-06-01', limit=500).df
        stock['symbol'] = symbol
        stocks.append(stock)
    except Exception as e:
        continue

In [12]:
stocks = pd.concat(stocks)

In [ ]:
stocks.reset_index(inplace=True)

In [ ]:
stocks.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']
stocks.set_index('symbol', inplace=True)
stocks.date = pd.to_datetime(stocks.date, utc=True)

In [14]:
market = alpaca.polygon.historic_agg_v2('SPY', 1, 'day',
                                        _from='2015-01-01', to='2021-06-01').df
market['symbol'] = 'SPY'
market.reset_index(inplace=True)
market.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']
market.set_index('symbol', inplace=True)
market.date = pd.to_datetime(market.date, utc=True)

In [15]:
open_positions = {position.symbol: position for position in alpaca.list_positions()}

In [16]:
def buy_stock(symbol, qty):
    try:
        alpaca.submit_order(
            symbol=symbol,
            side='buy',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to buy {qty} shares of {symbol}', e)  

In [17]:
def sell_stock(symbol, qty):
    try:
        alpaca.submit_order(
            symbol=symbol,
            side='sell',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to sell {qty} shares of {symbol}', e)

In [18]:
alpaca.cancel_all_orders()

stocks_by_symbol = stocks.groupby('symbol')

# ci.compute(market)

for symbol in tqdm(symbols):
    if symbol in stocks.index: 
        stock = stocks_by_symbol.get_group(symbol)
        ci.compute(stock)
#         stock = stock.merge(market, on='date', suffixes=['', '_market'])
        prediction, last_close = predict(stock, symbol)
        print(symbol, prediction)
        if (prediction >= 0.5) and (symbol not in open_positions):
            qty = 1000 // last_close
            buy_stock(symbol, qty)
        elif (prediction < 0.5) and (symbol in open_positions):
            qty = open_positions[symbol].qty
            sell_stock(symbol, qty)

FTAI 0.70341295
AAPL 0.59126234
MSFT 0.6532772
CSCO 0.6986551
INTC 0.66330177
ORCL 0.6718571
EBAY 0.6728165
BAC 0.70408285
NIO 0.789213
FB 0.6045616
SIRI 0.69577813
PAE 0
SNAP 0.7007701
UBER 0.7411993
GE 0.75611895
TWTR 0.663019
VALE 0.7357824
VXX 0.804323
F 0.70990527
KTB 0.72669417
AMAT 0.7018102
QCOM 0.6967875
NFLX 0.7273405
NOK 0.6962857
PFE 0.65809137
BABA 0.6548455
V 0.62550116
ZNGA 0.7283658
ITUB 0.7354381
BB 0.75351083
PBR 0.7274286
NVDA 0.64394116
SBUX 0.65700865
MU 0.73701084
GILD 0.74902207
HPQ 0.7280784
AMD 0.7011914
CMCSA 0.64943874
FCX 0.7486084
HPE 0.71727383
GM 0.71578974
CHK 0.36912838
BIDU 0.70765173
MA 0.6130262
T 0.6659573
JD 0.7467504
BBD 0.66718227
NKE 0.6004493
JPM 0.70364517
INFY 0.71378404
KMI 0.7389026
GRPN 0.7063054
ROKU 0.5503551
TJX 0.64525187
WORK 0.7625213
DAL 0.7779592
WFC 0.74401164
IQ 0.809083
MRVL 0.6256762
MS 0.7008719
IAA 0.66895044
PINS 0.66102076
LVS 0.71028715
AUY 0.7202972
TSM 0.69871485
SQ 0.72484046
JNPR 0.70412683
BSM 0.73074853
SLM 0.8026392